<a href="https://colab.research.google.com/github/CheilaBaiao/Pantanal/blob/main/1)_00_config_ingest_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Setup: instalar libs, montar Drive e configurar projeto (rodar 1x por sessão)
# Instalações essenciais (evite repetir em outras células)
%pip -q install numpy pandas geopandas shapely pyproj rasterio rioxarray xarray tqdm pyyaml joblib earthengine-api

# Imports base
import os, json, time, glob, math, gc, warnings, yaml
from pathlib import Path
import numpy as np, pandas as pd
from tqdm.notebook import tqdm

import geopandas as gpd
from shapely.geometry import box

# Montar Google Drive (Colab)
from google.colab import drive
drive.mount('/content/drive')

# --- PASTAS DO PROJETO ---
# Use nomes simples (sem subpastas no 'folder' do GEE) para evitar shards/caminhos esquisitos
BASE_DIR = Path("/content/drive/MyDrive/Pantanal_TippingPoints/index")
RAW_DIR  = BASE_DIR / "raw"
INT_DIR  = BASE_DIR / "interim"
OUT_DIR  = BASE_DIR / "outputs"
LOG_DIR  = BASE_DIR / "logs"
START_YEAR = 2009
END_YEAR   = 2021
for d in (RAW_DIR, INT_DIR, OUT_DIR, LOG_DIR): d.mkdir(parents=True, exist_ok=True)

# --- CONFIG ÚNICA (sem duplicatas) ---
CONFIG = {
    # Calendário/anos (mantém, mas será ignorado se "only_months" for usado)
    "calendar": "bimonthly",              # "monthly" ou "bimonthly"
    "years": list(range(START_YEAR, END_YEAR+1)),
    "only_months": None,


    # Projeção/escala
    "crs_epsg": 31983,                    # SIRGAS 2000 / UTM 23S
    "optical_pixel_m": 30,
    "agg_optical_to_m": 300,

    # Máscaras / NoData
    "nodata": -32768,
    "mask_water_threshold": 0.05,
    "min_obs_hint": 2,

    # Janela flex (DJFM mais larga)
    "wet_months": [12, 1, 2, 3],
    "wet_window_months": 4,
    "dry_window_months": 2,

    # Dados e flags
    "use_smap_vod": True,
    "resume": True,

    # Assets
    "ee_assets": {
        "landsat_collections": {
            "L5_SR": "LANDSAT/LT05/C02/T1_L2",
            "L7_SR": "LANDSAT/LE07/C02/T1_L2",
            "L8_SR": "LANDSAT/LC08/C02/T1_L2",
            "L9_SR": "LANDSAT/LC09/C02/T1_L2"
        },
        "smap_vod": "NASA_USDA/HSL/SMAP10KM_soil_moisture"
    },

    # Caminho do limite no Drive
    "boundary_path": "/content/drive/MyDrive/Pantanal_TippingPoints/Pantanal.shp",

    # Pastas de export no Drive
    "gee_drive_folder_optical": "Pantanal_TippingPoints_optical",
    "gee_drive_folder_vod":     "Pantanal_TippingPoints_vod"
}

# Helpers genéricos
def log(msg):
    print(time.strftime("[%Y-%m-%d %H:%M:%S]"), msg)

def get_time_slices(years, calendar="bimonthly", only_months=None):
    """Gera YYYYMM; se 'only_months' vier preenchido, usa só essa lista."""
    if only_months:
        return list(only_months)
    ts=[]
    for y in years:
        if calendar=="monthly":
            ts.extend([f"{y}{m:02d}" for m in range(1,13)])
        else:
            ts.extend([f"{y}{m:02d}" for m in (1,3,5,7,9,11)])
    return ts

# >>> use a lista limitada (se houver)
TIME_SLICES = get_time_slices(CONFIG["years"], CONFIG["calendar"], CONFIG.get("only_months"))
print("Períodos-alvo:", TIME_SLICES)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.4 MB/s eta 0:00:00
Mounted at /content/drive
Períodos-alvo: ['200901', '200903', '200905', '200907', '200909', '200911', '201001', '201003', '201005', '201007', '201009', '201011', '201101', '201103', '201105', '201107', '201109', '201111', '201201', '201203', '201205', '201207', '201209', '201211', '201301', '201303', '201305', '201307', '201309', '201311', '201401', '201403', '201405', '201407', '201409', '201411', '201501', '201503', '201505', '201507', '201509', '201511', '201601', '201603', '201605', '201607', '201609', '201611', '201701', '201703', '201705', '201707', '201709', '201711', '201801', '201803', '201805', '201807', '201809', '201811', '201901', '201903', '201905', '201907', '201909', '201911', '202001', '202003', '202005', '202007', '202009', '202011', '202101', '202103', '202105', '202107', '202109', '202111']


In [ ]:
# @title Célula 3 — Carregar boundary do Drive → ee.Geometry (pant) e pant_simpl
import json
from pathlib import Path
import geopandas as gpd
from shapely.geometry import MultiPolygon, Polygon
from shapely.ops import unary_union

# (opcional) parâmetro de tolerância para a simplificação (em metros no CRS do projeto)
SIMPLIFY_TOL_M = 500  # ajuste se quiser; 300–1000 m costuma ir bem para region

bpath = Path(CONFIG["boundary_path"])
assert bpath.exists(), f"Limite não encontrado: {bpath}. Ajuste CONFIG['boundary_path']."

# 1) Ler e normalizar
gdf = gpd.read_file(bpath)
assert len(gdf), f"Nenhuma feição encontrada em {bpath}"

# Corrige geometrias inválidas (buffer(0)) e remove vazios
gdf["geometry"] = gdf["geometry"].buffer(0)
gdf = gdf[~gdf.geometry.is_empty & gdf.geometry.notnull()].copy()
assert len(gdf), "Todas as geometrias ficaram vazias após correção."

# 2) Dissolver em um único polígono
if "name" in gdf.columns:
    gdf_diss = gdf.dissolve()
else:
    gdf["_ones_"] = 1
    gdf_diss = gdf.dissolve("_ones_").drop(columns="_ones_", errors="ignore")

geom_proj = gdf_diss.geometry.values[0]
assert geom_proj.is_valid and not geom_proj.is_empty, "Geometry inválida após dissolve."

# 3) Reprojetar para WGS84 (GEE espera lon/lat em EPSG:4326 nas coords do 'region')
gdf_wgs = gpd.GeoDataFrame(geometry=[geom_proj], crs=gdf.crs).to_crs(4326)

# 4) Exportar referência em GeoJSON (WGS84)
ref_dir = (INT_DIR / "boundaries"); ref_dir.mkdir(parents=True, exist_ok=True)
ref_geojson_fp = ref_dir / "pantanal_boundary_wgs84.geojson"
gdf_wgs.to_file(ref_geojson_fp, driver="GeoJSON")
print("✔ Boundary WGS84 salvo em:", ref_geojson_fp)

# 5) Construir ee.Geometry (aceita MultiPolygon também)
geojson = json.loads(gdf_wgs.to_json())
features = geojson["features"]
assert features, "GeoJSON sem features após reprojeção."

# Garante MultiPolygon/Polygon “limpo”
def _as_geom_coords(feat_geom):
    gtype = feat_geom.get("type","")
    if gtype == "Polygon":
        return [feat_geom["coordinates"]]
    elif gtype == "MultiPolygon":
        return feat_geom["coordinates"]
    else:
        raise ValueError(f"Tipo de geometria não suportado: {gtype}")

all_mparts = []
for feat in features:
    all_mparts.extend(_as_geom_coords(feat["geometry"]))

# Reconstrói como MultiPolygon → ee.Geometry
if len(all_mparts) == 1:
    pant = ee.Geometry.Polygon(all_mparts[0], proj=None, geodesic=True, evenOdd=True)
else:
    pant = ee.Geometry.MultiPolygon(all_mparts, proj=None, geodesic=True, evenOdd=True)

# 6) Versão simplificada apenas para usar como 'region' nos exports
#    A simplificação é feita no CRS do projeto (métrico), para tolerância em metros.
gdf_metric = gdf_wgs.to_crs(CONFIG["crs_epsg"])
geom_metric = gdf_metric.geometry.values[0].simplify(SIMPLIFY_TOL_M, preserve_topology=True)
geom_metric = geom_metric.buffer(0)  # reforça validade pós-simplificação
gdf_metric_simpl = gpd.GeoDataFrame(geometry=[geom_metric], crs=CONFIG["crs_epsg"]).to_crs(4326)

geojson_simpl = json.loads(gdf_metric_simpl.to_json())
parts_simpl = _as_geom_coords(geojson_simpl["features"][0]["geometry"])
if len(parts_simpl) == 1:
    pant_simpl = ee.Geometry.Polygon(parts_simpl[0], proj=None, geodesic=True, evenOdd=True)
else:
    pant_simpl = ee.Geometry.MultiPolygon(parts_simpl, proj=None, geodesic=True, evenOdd=True)
pant_bbox = pant_simpl.bounds(1)
# 7) Prints de sanidade
#    (área aproximada esférica em km², usando ee.Geometry area em m²)
area_full_km2  = pant.area(maxError=10).getInfo() / 1e6
area_simpl_km2 = pant_simpl.area(maxError=10).getInfo() / 1e6
print(f"✔ ee.Geometry criado. Área ~ full: {area_full_km2:,.1f} km² | simpl: {area_simpl_km2:,.1f} km²")
print(f"✔ pant e pant_simpl prontos (tolerância {SIMPLIFY_TOL_M} m).")


✔ Boundary WGS84 salvo em: /content/drive/MyDrive/Pantanal_TippingPoints/index/interim/boundaries/pantanal_boundary_wgs84.geojson
✔ ee.Geometry criado. Área ~ full: 151,438.0 km² | simpl: 151,463.5 km²
✔ pant e pant_simpl prontos (tolerância 500 m).


In [ ]:
# @title 3.LANDSAT — Helpers Landsat L2 (escala, máscara) + índices NDVI/EVI/NBR/MNDWI
import ee

def sr_scale(img):
    """
    Aplica a escala dos produtos Landsat Collection 2 Level-2 (reflectância):
      SR = DN * 0.0000275 - 0.2
    Mantém QA_PIXEL/QA_RADSAT.
    """
    optical = img.select('SR_B.*').multiply(0.0000275).add(-0.2)
    qa = img.select('QA_PIXEL')
    return img.addBands(optical, None, True).addBands(qa)

def cloud_mask_l2(img):
    """
    Máscara de nuvem/sombra/neve + saturação radiométrica.
    Bits em QA_PIXEL (C2 L2):
      3: Cloud, 4: Shadow, 5: Snow
    QA_RADSAT: saturação em alguma banda.
    Além disso, restringe a faixa física após escala (~[-0.2, 1.0]).
    """
    qa = img.select('QA_PIXEL')
    cloud  = qa.bitwiseAnd(1 << 3).neq(0)
    shadow = qa.bitwiseAnd(1 << 4).neq(0)
    snow   = qa.bitwiseAnd(1 << 5).neq(0)
    mask_clouds = cloud.Or(shadow).Or(snow)

    # qualquer banda saturada?
    sat_any = img.select('QA_RADSAT').reduce(ee.Reducer.max()).gt(0)

    # faixa física pós-escala
    sr = img.select('SR_B.*')
    valid_low  = sr.reduce(ee.Reducer.min()).gt(-0.199)  # folga
    valid_high = sr.reduce(ee.Reducer.max()).lt(1.001)

    return (img
            .updateMask(mask_clouds.Not())
            .updateMask(sat_any.Not())
            .updateMask(valid_low.And(valid_high)))

def add_indices_for(sensor_key):
    """
    Calcula NDVI, EVI, NBR e MNDWI com mapeamento de bandas por sensor.
    Saídas clampadas a intervalos físicos para evitar estouros mais à frente.
    """
    def _fn(img):
        if sensor_key in ("L5_SR", "L7_SR"):
            b = {'BLUE':'SR_B1','GREEN':'SR_B2','RED':'SR_B3','NIR':'SR_B4','SWIR1':'SR_B5','SWIR2':'SR_B7'}
        else:  # L8_SR, L9_SR
            b = {'BLUE':'SR_B2','GREEN':'SR_B3','RED':'SR_B4','NIR':'SR_B5','SWIR1':'SR_B6','SWIR2':'SR_B7'}

        nir   = img.select(b['NIR'])
        red   = img.select(b['RED'])
        blue  = img.select(b['BLUE'])
        swir1 = img.select(b['SWIR1'])
        swir2 = img.select(b['SWIR2'])

        # NDVI
        ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
        # EVI (com denominador protegido)
        den  = nir.add(red.multiply(6)).subtract(blue.multiply(7.5)).add(1)
        evi  = nir.subtract(red).multiply(2.5).divide(den.max(0.05)).rename('EVI')
        # NBR
        nbr  = nir.subtract(swir2).divide(nir.add(swir2)).rename('NBR')
        # MNDWI (usando GREEN e SWIR1)
        mndwi = img.expression('(g - s1) / (g + s1)', {'g': img.select(b['GREEN']), 's1': swir1}).rename('MNDWI')

        # clamp físico (evita estouro ao quantizar ×10000 depois)
        ndvi  = ndvi.max(-1).min(1)
        evi   = evi.max(-1.5).min(1.5)
        nbr   = nbr.max(-1).min(1)
        mndwi = mndwi.max(-1).min(1)

        return img.addBands([ndvi, evi, nbr, mndwi], overwrite=True)
    return _fn

def landsat_ic(sensor_key):
    """
    Retorna a ImageCollection Landsat C2 L2 do sensor, já com:
      - escala L2 aplicada
      - máscara de nuvem/sombra/saturação/faixa
      - bandas de índices NDVI, EVI, NBR, MNDWI calculadas
    """
    coll_id = CONFIG["ee_assets"]["landsat_collections"][sensor_key]
    coll = ee.ImageCollection(coll_id)
    return (coll
            .map(sr_scale)
            .map(cloud_mask_l2)
            .map(add_indices_for(sensor_key)))

print("✔ Célula 3.LANDSAT pronta: sr_scale, cloud_mask_l2, add_indices_for, landsat_ic.")


✔ Célula 3.LANDSAT pronta: sr_scale, cloud_mask_l2, add_indices_for, landsat_ic.


In [ ]:
# @title Célula 4 — Compósito 30 m (estrito) + métricas 300 m (inclui MNDWI/dNBR/RdNBR)
import ee

def ensure_default_proj(img, scale_m=None):
    """Garante projeção default antes de reduceResolution."""
    if scale_m is None:
        scale_m = CONFIG['optical_pixel_m']  # 30 m
    proj = ee.Projection(f"EPSG:{CONFIG['crs_epsg']}").atScale(scale_m)
    return img.setDefaultProjection(proj)

def build_comp_for_export_strict(yyyymm: str, region=None) -> ee.Image:
    """
    Igual à versão original, mas filtra e clippa logo no início para o 'region'
    (tile). Isso reduz muito o volume de dados e evita OOM.
    """
    if region is None:
        region = pant  # retrocompatível

    start, end = period_for_yyyymm(yyyymm)

    # Filtra por tile (region) — MUITO importante para performance
    ic = (
        landsat_ic('L5_SR').filterDate(start, end).filterBounds(region)
        .merge(landsat_ic('L7_SR').filterDate(start, end).filterBounds(region))
        .merge(landsat_ic('L8_SR').filterDate(start, end).filterBounds(region))
        .merge(landsat_ic('L9_SR').filterDate(start, end).filterBounds(region))
        .map(lambda im: im.select(['NDVI','EVI','NBR','MNDWI']).clip(region))
    )

    comp = ic.reduce(ee.Reducer.median()).rename(['NDVI','EVI','NBR','MNDWI']).toFloat()
    obs  = ic.select('NDVI').count().rename('OBS')

    water_keep = comp.select('MNDWI').lte(CONFIG['mask_water_threshold'])
    valid_all  = comp.mask().reduce(ee.Reducer.min())
    has_obs    = obs.gte(1)
    common_mask = water_keep.And(valid_all).And(has_obs)

    comp_m = comp.updateMask(common_mask).clip(region)
    obs_m  =  obs.updateMask(common_mask).clip(region)
    return comp_m.addBands(obs_m)


def prev_yyyymm(yyyymm: str):
    try:
        i = TIME_SLICES.index(yyyymm)
        return TIME_SLICES[i-1] if i > 0 else None
    except Exception:
        return None

def build_fire_metrics_300m(yyyymm: str, region=None) -> ee.Image:
    """
    Igual ao original, porém:
      - usa compósitos 30 m já CLIPPADOS ao tile (region)
      - reduz para 300 m depois do clip, diminuindo MUITO a carga
    Conteúdo (bandas/valores) permanece igual dentro do tile.
    """
    if region is None:
        region = pant

    yprev = prev_yyyymm(yyyymm)
    comp_cur_30 = ensure_default_proj(build_comp_for_export_strict(yyyymm, region), CONFIG['optical_pixel_m'])

    if yprev is None:
        idx30 = comp_cur_30.select(['NDVI','EVI','NBR','MNDWI'])
        obs30 = comp_cur_30.select('OBS').toFloat()
        reducer = ee.Reducer.mean()

        idx_agg = (idx30
                   .reduceResolution(reducer=reducer, maxPixels=4096)
                   .reproject(crs=f"EPSG:{CONFIG['crs_epsg']}", scale=CONFIG['agg_optical_to_m'])
                   .clip(region))
        obs_agg = (obs30
                   .reduceResolution(reducer=reducer, maxPixels=4096)
                   .reproject(crs=f"EPSG:{CONFIG['crs_epsg']}", scale=CONFIG['agg_optical_to_m'])
                   .clip(region))

        empty = ee.Image(0).updateMask(ee.Image(0))
        dnbr_empty  = empty.rename('dNBR')
        rdnbr_empty = empty.rename('RdNBR')

        return ee.Image.cat([
            idx_agg.select(['NDVI','EVI','NBR','MNDWI']),
            dnbr_empty, rdnbr_empty,
            obs_agg.rename('OBS')
        ]).toFloat().clip(region)

    comp_pre_30 = ensure_default_proj(build_comp_for_export_strict(yprev, region), CONFIG['optical_pixel_m'])
    nbr_pre = comp_pre_30.select('NBR')
    nbr_pos = comp_cur_30.select('NBR')

    dnbr_30  = nbr_pre.subtract(nbr_pos).rename('dNBR')
    eps  = ee.Image.constant(0.1)
    denom = nbr_pre.abs().sqrt().max(eps)
    rdnbr_30 = dnbr_30.divide(denom).rename('RdNBR')

    reducer = ee.Reducer.mean()

    idx_agg = (comp_cur_30.select(['NDVI','EVI','NBR','MNDWI'])
               .reduceResolution(reducer=reducer, maxPixels=4096)
               .reproject(crs=f"EPSG:{CONFIG['crs_epsg']}", scale=CONFIG['agg_optical_to_m'])
               .clip(region))
    dnbr_agg = (dnbr_30
                .reduceResolution(reducer=reducer, maxPixels=4096)
                .reproject(crs=f"EPSG:{CONFIG['crs_epsg']}", scale=CONFIG['agg_optical_to_m'])
                .clip(region))
    rdnbr_agg = (rdnbr_30
                 .reduceResolution(reducer=reducer, maxPixels=4096)
                 .reproject(crs=f"EPSG:{CONFIG['crs_epsg']}", scale=CONFIG['agg_optical_to_m'])
                 .clip(region))
    obs_agg = (comp_cur_30.select('OBS').toFloat()
               .reduceResolution(reducer=reducer, maxPixels=4096)
               .reproject(crs=f"EPSG:{CONFIG['crs_epsg']}", scale=CONFIG['agg_optical_to_m'])
               .clip(region))

    return ee.Image.cat([
        idx_agg.select(['NDVI','EVI','NBR','MNDWI']),
        dnbr_agg, rdnbr_agg,
        obs_agg.rename('OBS')
    ]).toFloat().clip(region)


# (opcional) smoke-test rápido:
_test = build_comp_for_export_strict(TIME_SLICES[0])
print("✔ Célula 4 pronta: build_comp_for_export_strict e build_fire_metrics_300m definidas.")


✔ Célula 4 pronta: build_comp_for_export_strict e build_fire_metrics_300m definidas.


In [ ]:
# === Célula 5 — Export 300 m (rodar por ANO, com resume e re-sync) ===
# - Overlap + reduce→clip (evita costuras)
# - Grade de tiles em UTM (retângulos planos, geodesic=False)
# - Re-sync do checkpoint com o DISCO (single/merged) antes de exportar
# - Lança somente peças faltantes (resume via dedupe pNN)
import json, time, glob, re
from pathlib import Path
from tqdm.auto import tqdm
import ee

# ---------------- Parâmetros gerais ----------------
COG               = False
EXPORT_SCALE_M    = int(CONFIG['agg_optical_to_m'])   # 300 m
NODATA_INT16      = int(CONFIG['nodata'])             # -32768
OPT_FOLDER        = CONFIG["gee_drive_folder_optical"]

QUEUE_SIZE        = 2
SLEEP_S           = 10
OVERLAP_M         = 600        # >= 1 célula de 300 m é suficiente

# >>> Escolha dos anos a rodar (edite aqui)
YEARS_TO_RUN = [2020]          # ex.: list(range(2005, 2022))

# ---------------- Projeções/helpers ----------------
PROJ_UTM = ee.Projection(f"EPSG:{CONFIG['crs_epsg']}")

try:
    REGION_EXPORT_UTM = pant.transform(PROJ_UTM, 1).bounds(1, PROJ_UTM)
except NameError:
    REGION_EXPORT_UTM = pant_simpl.transform(PROJ_UTM, 1).bounds(1, PROJ_UTM)

def _proj_30m():
    return ee.Projection(f"EPSG:{CONFIG['crs_epsg']}").atScale(CONFIG['optical_pixel_m'])

def _proj_300m():
    return ee.Projection(f"EPSG:{CONFIG['crs_epsg']}").atScale(EXPORT_SCALE_M)

def _reduce_to_300m(img, region):
    img = img.setDefaultProjection(_proj_30m())
    return (img
            .reduceResolution(reducer=ee.Reducer.mean(), maxPixels=4096)
            .reproject(crs=_proj_300m())
            .clip(region))

def _build_out_image_from_globals(comp_cur_30, dnbr_30, rdnbr_30, region_reduce, region_clip):
    comp_cur_30 = comp_cur_30.setDefaultProjection(_proj_30m())
    dnbr_30     = dnbr_30.setDefaultProjection(_proj_30m())
    rdnbr_30    = rdnbr_30.setDefaultProjection(_proj_30m())

    idx30 = comp_cur_30.select(['NDVI','EVI','NBR','MNDWI'])
    obs30 = comp_cur_30.select('OBS').toFloat()

    idx_agg   = _reduce_to_300m(idx30,   region_reduce)
    dnbr_agg  = _reduce_to_300m(dnbr_30, region_reduce)
    rdnbr_agg = _reduce_to_300m(rdnbr_30, region_reduce)
    obs_agg   = _reduce_to_300m(obs30,   region_reduce)

    out_float = ee.Image.cat([
        idx_agg.select(['NDVI','EVI','NBR','MNDWI']),
        dnbr_agg.rename('dNBR'),
        rdnbr_agg.rename('RdNBR'),
        obs_agg.rename('OBS')
    ]).toFloat()

    idx_names = ['NDVI','EVI','NBR','MNDWI','dNBR','RdNBR']
    idx_i16 = out_float.select(idx_names).multiply(10000).round().toInt16()
    obs_i16 = out_float.select('OBS').round().toInt16()
    nod = ee.Image.constant(NODATA_INT16).toInt16()

    out = idx_i16.unmask(nod).addBands(obs_i16.unmask(nod)).reproject(_proj_300m())
    return out.clip(region_clip)

# ---------------- Grade UTM ----------------
def _make_grid(region, nx=15, ny=15):
    reg_utm = ee.Geometry(region).transform(PROJ_UTM, 1)
    b = reg_utm.bounds(1, PROJ_UTM)
    coords = ee.List(b.coordinates().get(0))
    ll = ee.List(coords.get(0)); ur = ee.List(coords.get(2))
    xmin = ee.Number(ll.get(0)); ymin = ee.Number(ll.get(1))
    xmax = ee.Number(ur.get(0)); ymax = ee.Number(ur.get(1))
    dx = xmax.subtract(xmin).divide(nx); dy = ymax.subtract(ymin).divide(ny)
    tiles=[]
    for iy in range(ny):
        for ix in range(nx):
            x0 = xmin.add(dx.multiply(ix)); x1 = x0.add(dx)
            y0 = ymin.add(dy.multiply(iy)); y1 = y0.add(dy)
            tiles.append(ee.Geometry.Rectangle([x0,y0,x1,y1], PROJ_UTM, False))
    return tiles

# ---------------- Utilidades de Drive/estado ----------------
ROOTS = [Path("/content/drive/MyDrive"), Path("/content/drive/My Drive")]
def _glob_all(pat):
    s=set()
    for r in ROOTS: s |= set(glob.glob(str(r/pat)))
    return sorted(s)

def _month_has_single(ym):   # opt_YYYYMM.tif
    return bool(_glob_all(f"{OPT_FOLDER}/opt_{ym}.tif"))

def _month_has_merged(ym):   # ..._merged/opt_YYYYMM_merged.tif
    return bool(_glob_all(f"{OPT_FOLDER}_merged/opt_{ym}_merged.tif"))

def _files_for_month(ym):
    single = _glob_all(f"{OPT_FOLDER}/opt_{ym}.tif")
    merged = _glob_all(f"{OPT_FOLDER}_merged/opt_{ym}_merged.tif")
    pieces = _glob_all(f"{OPT_FOLDER}/opt_{ym}_p*.tif")
    return single, merged, pieces

def _existing_piece_indices(ym):
    # dedupe: reconhece pNN e ignora duplicatas com " (1)" etc.
    rx = re.compile(rf"^opt_{ym}_p(\d+)(?: \(\d+\))?\.tif$", re.IGNORECASE)
    idx=set()
    for fp in _glob_all(f"{OPT_FOLDER}/opt_{ym}_p*.tif"):
        m = rx.match(Path(fp).name)
        if m: idx.add(int(m.group(1)))
    return idx

def _unique_piece_count(ym):
    return len(_existing_piece_indices(ym))

CKP_OPT = LOG_DIR / "export_optical_checkpoint.json"
state_opt = json.load(open(CKP_OPT)) if CKP_OPT.exists() else {"done": {}}
def _save_ckp_opt(): json.dump(state_opt, open(CKP_OPT, "w"))

# ---------------- Montagem por mês (split 15×15 sempre) ----------------
def _build_out_image_whole_region(yyyymm: str, region):
    nx, ny = 15, 15
    tiles  = _make_grid(region, nx=nx, ny=ny)
    existing = _existing_piece_indices(yyyymm)
    launched = 0
    yprev = prev_yyyymm(yyyymm)

    for k, q in enumerate(tiles):
        if k in existing:
            continue  # peça já existe (mesmo que duplicada, dedupe é no merge)

        q_pad = ee.Geometry(q).buffer(OVERLAP_M)

        comp_cur_30_q = ensure_default_proj(
            build_comp_for_export_strict(yyyymm, region=q_pad),
            CONFIG['optical_pixel_m']
        ).setDefaultProjection(_proj_30m())

        if yprev:
            comp_pre_30_q = ensure_default_proj(
                build_comp_for_export_strict(yprev, region=q_pad),
                CONFIG['optical_pixel_m']
            ).setDefaultProjection(_proj_30m())
            dnbr_30_q  = comp_pre_30_q.select('NBR').subtract(comp_cur_30_q.select('NBR')).rename('dNBR')
            denom_q    = comp_pre_30_q.select('NBR').abs().sqrt().max(ee.Image.constant(0.1))
            rdnbr_30_q = dnbr_30_q.divide(denom_q).rename('RdNBR')
        else:
            empty_q = ee.Image(0).updateMask(ee.Image(0)).setDefaultProjection(_proj_30m())
            dnbr_30_q, rdnbr_30_q = empty_q.rename('dNBR'), empty_q.rename('RdNBR')

        out_q = _build_out_image_from_globals(comp_cur_30_q, dnbr_30_q, rdnbr_30_q, q_pad, q)

        desc = f"opt_{yyyymm}_p{k:02d}"
        task = ee.batch.Export.image.toDrive(
            image = out_q,
            description   = desc,
            folder        = OPT_FOLDER,
            fileNamePrefix= desc,
            region        = q,
            crs           = f"EPSG:{CONFIG['crs_epsg']}",
            scale         = EXPORT_SCALE_M,
            maxPixels     = 1e13,
            fileFormat    = 'GeoTIFF',
            formatOptions = {'noData': NODATA_INT16}
        )
        task.start()
        launched += 1

    log(f"[{yyyymm}] tiles lançados agora: {launched} (faltantes).")
    return None

def _launch_export_month(yyyymm: str, region):
    # se já existe single ou merged → marca done e pula
    if _month_has_single(yyyymm) or _month_has_merged(yyyymm):
        state_opt["done"][yyyymm] = True
        return None
    return _build_out_image_whole_region(yyyymm, region)

# ---------------- Scheduler por ANO (com re-sync embutido) ----------------
def _time_slices_for_year(year: int):
    if CONFIG["calendar"] == "monthly":
        return [f"{year}{m:02d}" for m in range(1,13)]
    else:
        return [f"{year}{m:02d}" for m in (1,3,5,7,9,11)]

EXPECTED_PIECES_DEFAULT = 225

for Y in YEARS_TO_RUN:
    months = _time_slices_for_year(Y)

    # (1) Re-sync do checkpoint: done=True somente se existe single OU merged no disco
    changed=0
    for ym in months:
        single, merged, _ = _files_for_month(ym)
        new_done = bool(single or merged)
        if state_opt["done"].get(ym, False) != new_done:
            state_opt["done"][ym] = new_done
            changed += 1
    if changed:
        _save_ckp_opt(); print(f"✔ {Y}: checkpoint re-sincronizado ({changed} updates).")
    else:
        print(f"✔ {Y}: checkpoint já consistente.")

    # (2) Diagnóstico: meses faltando (sem single/merged) e quantas peças únicas tem
    faltando=[]
    for ym in months:
        if not state_opt["done"].get(ym, False):
            nuniq = _unique_piece_count(ym)
            faltando.append((ym, nuniq, EXPECTED_PIECES_DEFAULT - nuniq))

    if faltando:
        print(f"→ {Y}: meses faltando (únicas / faltam):")
        for ym, nuniq, miss in faltando:
            print(f"   {ym}: {nuniq} / {miss}")
    else:
        print(f"→ {Y}: nenhum mês faltando (todos com single/merged).")

    # (3) Exporta SOMENTE os que estão faltando (resume garante que só peças ausentes são lançadas)
    to_run = [ym for ym,_n,_miss in faltando]
    for ym in to_run:
        _launch_export_month(ym, REGION_EXPORT_UTM)

print("✔ Exports disparados para:", YEARS_TO_RUN)
print(f"✔ Checkpoint em: {CKP_OPT}")



✔ 2020: checkpoint re-sincronizado (5 updates).
→ 2020: meses faltando (únicas / faltam):
   202003: 129 / 96
   202005: 0 / 225
   202007: 0 / 225
   202009: 0 / 225
   202011: 0 / 225
[2025-10-27 11:25:19] [202003] tiles lançados agora: 96 (faltantes).
[2025-10-27 11:57:18] [202005] tiles lançados agora: 225 (faltantes).
[2025-10-27 12:29:11] [202007] tiles lançados agora: 225 (faltantes).
[2025-10-27 13:00:58] [202009] tiles lançados agora: 225 (faltantes).
[2025-10-27 13:32:54] [202011] tiles lançados agora: 225 (faltantes).
✔ Exports disparados para: [2020]
✔ Checkpoint em: /content/drive/MyDrive/Pantanal_TippingPoints/index/logs/export_optical_checkpoint.json


In [ ]:
# === Célula 5B — Merge anual com dedupe de pNN (rodar após saírem os patches) ===
import os, re, glob, json, time, shutil, tempfile
from pathlib import Path
import rasterio as rio
from rasterio.merge import merge
from tqdm.auto import tqdm

OPT_FOLDER   = CONFIG["gee_drive_folder_optical"]
NODATA_I16   = int(CONFIG.get("nodata", -32768))
MERGE_DIR    = Path("/content/drive/MyDrive") / f"{OPT_FOLDER}_merged"
MERGE_DIR.mkdir(parents=True, exist_ok=True)

EXPECTED_PIECES_DEFAULT   = 225
EXPECTED_PIECES_PER_MONTH = {}  # ex.: {"201709": 196}

# >>> Anos a mesclar (mesmos da célula 5)
YEARS_TO_RUN = [2019,2020,2021]

ROOTS = [Path("/content/drive/MyDrive"), Path("/content/drive/My Drive")]
def _glob_all(pat):
    s=set()
    for r in ROOTS: s |= set(glob.glob(str(r / pat)))
    return sorted(s)

def _files_for_month(ym):
    single = _glob_all(f"{OPT_FOLDER}/opt_{ym}.tif")
    merged = _glob_all(f"{OPT_FOLDER}_merged/opt_{ym}_merged.tif")
    pieces = _glob_all(f"{OPT_FOLDER}/opt_{ym}_p*.tif")
    return single, merged, pieces

def _validate_merged(fp):
    try:
        with rio.open(fp) as src:
            ok = (src.count >= 7 and src.dtypes[0] == "int16" and
                  str(src.crs) == f"EPSG:{CONFIG['crs_epsg']}" and (src.nodata == NODATA_I16))
            _ = src.bounds
        return ok
    except Exception:
        return False

def _pick_one_per_index(ym, pieces):
    # dedupe: opt_YYYYMM_pNN.tif e variações " (1)"
    rx = re.compile(rf"^opt_{ym}_p(\d+)(?: \(\d+\))?\.tif$", re.IGNORECASE)
    buckets = {}
    for fp in pieces:
        m = rx.match(Path(fp).name)
        if not m: continue
        idx = int(m.group(1))
        st  = Path(fp).stat()
        cand = (fp, st.st_size, st.st_mtime)
        if idx not in buckets or (cand[1], cand[2]) > (buckets[idx][1], buckets[idx][2]):
            buckets[idx] = cand
    return [buckets[k][0] for k in sorted(buckets.keys())]

def _merge_month(ym, piece_fps):
    out_fp = MERGE_DIR / f"opt_{ym}_merged.tif"
    # se já existe e está íntegro, mantém
    if out_fp.exists() and _validate_merged(out_fp):
        return out_fp

    srcs = [rio.open(fp) for fp in piece_fps]
    crs0, dtype0, count0 = srcs[0].crs, srcs[0].dtypes[0], srcs[0].count
    for s in srcs[1:]:
        if s.crs != crs0:     raise ValueError(f"CRS difere em {s.name}")
        if s.count != count0: raise ValueError(f"Nº de bandas difere em {s.name}")
    mosaic, out_transform = merge(srcs, nodata=NODATA_I16, method='first')
    for s in srcs: s.close()

    meta = {"driver":"GTiff","height":mosaic.shape[1],"width":mosaic.shape[2],
            "transform":out_transform,"count":count0,"dtype":dtype0,"crs":crs0,
            "nodata":NODATA_I16,"tiled":True,"blockxsize":512,"blockysize":512,"BIGTIFF":"IF_SAFER"}
    with rio.open(out_fp, "w", **meta) as dst:
        dst.write(mosaic)
    return out_fp

def _months_for_year(year: int):
    if CONFIG["calendar"] == "monthly":
        return [f"{year}{m:02d}" for m in range(1,13)]
    else:
        return [f"{year}{m:02d}" for m in (1,3,5,7,9,11)]

CKP_OPT = LOG_DIR / "export_optical_checkpoint.json"
state_opt = json.load(open(CKP_OPT)) if CKP_OPT.exists() else {"done": {}}

merged_count = skipped = 0

for Y in YEARS_TO_RUN:
    months = _months_for_year(Y)
    for ym in tqdm(months, desc=f"Merge {Y}", dynamic_ncols=True):
        single, merged, pieces = _files_for_month(ym)

        # se já tem merged íntegro → skip
        if merged and _validate_merged(merged[0]):
            state_opt["done"][ym] = True
            skipped += 1
            continue

        # se tem arquivo único de mês inteiro → skip (considera pronto)
        if single:
            state_opt["done"][ym] = True
            skipped += 1
            continue

        # se não tem peças → ainda não terminou export
        if not pieces:
            continue

        # dedupe pNN
        chosen = _pick_one_per_index(ym, pieces)
        if not chosen:
            continue

        # garantia de completude (225 ou específico por mês)
        expected = EXPECTED_PIECES_PER_MONTH.get(ym, EXPECTED_PIECES_DEFAULT)
        if len(chosen) < expected:
            # ainda faltam peças → não mergeia para evitar lacunas
            continue

        try:
            out_fp = _merge_month(ym, chosen)
            state_opt["done"][ym] = True
            merged_count += 1
        except Exception as e:
            print(f"⚠️ {ym}: falhou no merge — {e}")

# salva checkpoint atualizado
json.dump(state_opt, open(CKP_OPT, "w"))
print(f"✔ Merge concluído. Novos meses mesclados: {merged_count} | Pulados: {skipped}")
print(f"✔ Checkpoint atualizado em: {CKP_OPT}")
print(f"Destino dos mesclados: {MERGE_DIR}")






Merge 2019:   0%|          | 0/6 [00:00<?, ?it/s]

Merge 2020:   0%|          | 0/6 [00:00<?, ?it/s]

Merge 2021:   0%|          | 0/6 [00:00<?, ?it/s]

✔ Merge concluído. Novos meses mesclados: 2 | Pulados: 16
✔ Checkpoint atualizado em: /content/drive/MyDrive/Pantanal_TippingPoints/index/logs/export_optical_checkpoint.json
Destino dos mesclados: /content/drive/MyDrive/Pantanal_TippingPoints_optical_merged


In [ ]:
# === Listar corretamente SINGLE vs MERGED (ignorando patches pNN) ===
import re, glob
from pathlib import Path

OPT_FOLDER = CONFIG["gee_drive_folder_optical"]
ROOTS = [Path("/content/drive/MyDrive"), Path("/content/drive/My Drive")]

# (opcional) filtrar por ano
YEAR_MIN = None  # ex.: 1985
YEAR_MAX = None  # ex.: 2021

re_single = re.compile(r"^opt_(\d{6})\.tif$")
re_merged = re.compile(r"^opt_(\d{6})_merged\.tif$")
re_piece  = re.compile(r"^opt_(\d{6})_p\d+(?: \(\d+\))?\.tif$", re.IGNORECASE)

def _glob_all(pat):
    s=set()
    for r in ROOTS:
        s |= set(glob.glob(str(r/pat)))
    return sorted(s)

def _in_year_range(ym):
    y = int(ym[:4])
    if YEAR_MIN is not None and y < YEAR_MIN: return False
    if YEAR_MAX is not None and y > YEAR_MAX: return False
    return True

# Coleta singles (apenas opt_YYYYMM.tif)
single_files = []
for fp in _glob_all(f"{OPT_FOLDER}/opt_*.tif"):
    name = Path(fp).name
    m = re_single.match(name)
    if m and _in_year_range(m.group(1)):
        single_files.append(fp)

# Coleta merged (suportando duas convenções de pasta)
merged_files = []
for merged_dir in (f"{OPT_FOLDER}_merged", OPT_FOLDER.replace("_optical","") + "_merged"):
    for fp in _glob_all(f"{merged_dir}/opt_*_merged.tif"):
        name = Path(fp).name
        m = re_merged.match(name)
        if m and _in_year_range(m.group(1)):
            merged_files.append(fp)

# Extrai meses
single_months = sorted({re_single.match(Path(fp).name).group(1) for fp in single_files})
merged_months = sorted({re_merged.match(Path(fp).name).group(1) for fp in merged_files})

# Conjuntos
S = set(single_months); M = set(merged_months)
both        = sorted(S & M)
only_single = sorted(S - M)
only_merged = sorted(M - S)

print("=== RESUMO (corrigido) ===")
print(f"Singles (opt_YYYYMM.tif) : {len(single_months)} meses")
print(f"Merged  (opt_YYYYMM_merged.tif): {len(merged_months)} meses")
print(f"Ambos   : {len(both)} | Apenas SINGLE: {len(only_single)} | Apenas MERGED: {len(only_merged)}")
print()

def _fmt(lst): return " ".join(lst) if lst else "(nenhum)"

print("— Meses com SINGLE:")
print(_fmt(single_months))
print("\n— Meses com MERGED:")
print(_fmt(merged_months))
print("\n— Meses com AMBOS (SINGLE + MERGED):")
print(_fmt(both))
print("\n— Meses apenas SINGLE:")
print(_fmt(only_single))
print("\n— Meses apenas MERGED:")
print(_fmt(only_merged))

# (Opcional) Se quiser também ver quantos patches existem por mês:
def count_unique_patches(ym):
    rx = re.compile(rf"^opt_{ym}_p(\d+)(?: \(\d+\))?\.tif$", re.IGNORECASE)
    seen=set()
    for fp in _glob_all(f"{OPT_FOLDER}/opt_{ym}_p*.tif"):
        m = rx.match(Path(fp).name)
        if m: seen.add(int(m.group(1)))
    return len(seen)

# Exemplo: imprimir um resumo de patches para meses com MERGED
if merged_months:
    print("\n— Patches únicos (pNN) para meses com MERGED — amostra:")
    for ym in merged_months[:10]:
        print(ym, count_unique_patches(ym))



=== RESUMO (corrigido) ===
Singles (opt_YYYYMM.tif) : 131 meses
Merged  (opt_YYYYMM_merged.tif): 86 meses
Ambos   : 0 | Apenas SINGLE: 131 | Apenas MERGED: 86

— Meses com SINGLE:
198509 198603 198607 198609 198611 198701 198703 198705 198707 198709 198711 198801 198803 198805 198807 198809 198811 198901 198903 198905 198907 198909 198911 199001 199003 199005 199007 199009 199011 199101 199103 199105 199107 199109 199111 199201 199203 199205 199207 199209 199211 199301 199303 199305 199307 199309 199311 199401 199403 199405 199407 199409 199411 199501 199503 199505 199507 199509 199511 199601 199603 199605 199607 199609 199611 199701 199703 199705 199707 199709 199711 199801 199803 199805 199807 199809 199811 199901 199903 199905 199907 199909 199911 200001 200003 200005 200007 200009 200011 200105 200107 200109 200111 200203 200205 200207 200209 200211 200301 200303 200305 200307 200309 200311 200407 200409 200411 200507 200509 200511 200607 200609 200611 200707 200709 200711 200807 2

In [ ]:
# Contar patches pNN só em /content/drive/MyDrive/Pantanal_TippingPoints_optical
import re, glob
from pathlib import Path
from collections import defaultdict

BASE = Path("/content/drive/MyDrive/Pantanal_TippingPoints_optical")
EXPECTED = 225  # número esperado de peças pNN por mês

# Busca apenas dentro dessa pasta (inclui subpastas se houver)
cands = set(glob.glob(str(BASE / "opt_??????_p*.tif")))
cands |= set(glob.glob(str(BASE / "**/opt_??????_p*.tif"), recursive=True))

rx = re.compile(r"opt_(\d{6})_p(\d+)(?: \(\d+\))?\.tif$", re.IGNORECASE)

# dedupe por (YYYYMM, NN) escolhendo o maior tamanho/mtime
by_key = {}  # (yyyymm, nn) -> (fp, size, mtime)
for fp in cands:
    name = Path(fp).name
    m = rx.fullmatch(name)
    if not m:
        continue
    ym, nn = m.group(1), int(m.group(2))
    st = Path(fp).stat()
    cand = (fp, st.st_size, st.st_mtime)
    if (ym, nn) not in by_key or (cand[1], cand[2]) > (by_key[(ym, nn)][1], by_key[(ym, nn)][2]):
        by_key[(ym, nn)] = cand

# contagens
per_month = defaultdict(set)  # yyyymm -> {nn}
for (ym, nn) in by_key.keys():
    per_month[ym].add(nn)

per_year = defaultdict(int)
for ym, idxs in per_month.items():
    per_year[ym[:4]] += len(idxs)

print("=== Patches por ano ===")
for y in sorted(per_year):
    print(f"{y}: {per_year[y]}")

print("\n=== Patches por mês (amostra 12) ===")
for ym in sorted(per_month)[:]:
    print(f"{ym}: {len(per_month[ym])}")

# meses com menos de EXPECTED
faltando_225 = [ym for ym, idxs in sorted(per_month.items()) if len(idxs) < EXPECTED]
print(f"\nMeses com menos de {EXPECTED} patches: {len(faltando_225)}")
print(faltando_225[:30])

# (Opcional) mostrar índices faltantes para um mês específico:
# YM_CHECK = "200507"  # <- defina se quiser inspecionar
# if YM_CHECK in per_month:
#     have = per_month[YM_CHECK]
#     missing = sorted(set(range(1, EXPECTED+1)) - have)
#     print(f"{YM_CHECK}: faltam {len(missing)} ->", missing[:50], "…" if len(missing)>50 else "")
# else:
#     print(f"{YM_CHECK} não encontrado.")





=== Patches por ano ===
1985: 1090
1986: 449
2001: 450
2002: 225
2004: 675
2005: 675
2006: 675
2007: 675
2008: 675
2009: 675
2010: 675
2011: 675
2012: 675
2013: 1350
2014: 1350
2015: 1350
2016: 1350
2017: 1350
2018: 1350
2019: 1350
2020: 354
2021: 1350
2022: 699
2023: 922

=== Patches por mês (amostra 12) ===
198501: 225
198503: 223
198505: 214
198507: 204
198511: 224
198601: 224
198605: 225
200101: 225
200103: 225
200201: 225
200401: 225
200403: 225
200405: 225
200501: 225
200503: 225
200505: 225
200601: 225
200603: 225
200605: 225
200701: 225
200703: 225
200705: 225
200801: 225
200803: 225
200805: 225
200901: 225
200903: 225
200905: 225
201001: 225
201003: 225
201005: 225
201101: 225
201103: 225
201105: 225
201201: 225
201203: 225
201205: 225
201301: 225
201303: 225
201305: 225
201307: 225
201309: 225
201311: 225
201401: 225
201403: 225
201405: 225
201407: 225
201409: 225
201411: 225
201501: 225
201503: 225
201505: 225
201507: 225
201509: 225
201511: 225
201601: 225
201603: 225
20160

In [ ]:
# @title Célula 6.QC — QC leve dos exports 300 m (série completa + arquivos mesclados)
import re, glob, json, math
from pathlib import Path
import numpy as np
import rasterio as rio
from tqdm.auto import tqdm
import pandas as pd

OPT_FOLDER = CONFIG["gee_drive_folder_optical"]
WATER_THR  = CONFIG.get("mask_water_threshold", 0.05)
NODATA_I16 = int(CONFIG.get("nodata", -32768))

# ---------- utilidades de path ----------
ROOTS = [Path("/content/drive/MyDrive"), Path("/content/drive/My Drive")]
MERGED_HINTS = ("merged", "mosaic", "mos")  # padrões comuns de mescla

def _glob_all(pattern: str):
    fps = []
    for r in ROOTS:
        fps.extend(glob.glob(str(r / OPT_FOLDER / pattern)))
    return sorted(set(fps))

def _extract_yyyymm(fp: str):
    # tenta capturar 6 dígitos após "opt_"
    m = re.search(r"opt_(\d{6})", Path(fp).name)
    return m.group(1) if m else None

def _find_merged_file(yyyymm: str):
    """
    Heurística:
      1) opt_YYYYMM.tif (sem sufixo de tile) → preferido
      2) qualquer opt_YYYYMM* contendo 'merged'/'mosaic'/'mos'
      3) fallback: maior arquivo dentre opt_YYYYMM*.tif
    Retorna [fp] (lista com um único caminho) ou [] se nada encontrado.
    """
    # 1) exato, sem sufixo
    exact = _glob_all(f"opt_{yyyymm}.tif")
    if exact:
        return [exact[0]]

    # 2) padrões de mescla
    cands = _glob_all(f"opt_{yyyymm}*.tif")
    merged_like = [fp for fp in cands if any(h in Path(fp).stem.lower() for h in MERGED_HINTS)]
    if merged_like:
        # se houver mais de um, escolhe o maior (em bytes)
        merged_like.sort(key=lambda p: Path(p).stat().st_size, reverse=True)
        return [merged_like[0]]

    # 3) fallback: maior entre todos os opt_YYYYMM*.tif
    if cands:
        cands.sort(key=lambda p: Path(p).stat().st_size, reverse=True)
        return [cands[0]]

    return []

def _list_month_files_merged_first(yyyymm: str):
    return _find_merged_file(yyyymm)

# ---------- leitura por blocos ----------
def _iter_valid_block(src, window):
    nd = src.nodata
    b1 = src.read(1, window=window).astype(np.int16)  # NDVI
    b2 = src.read(2, window=window).astype(np.int16)  # EVI
    b3 = src.read(3, window=window).astype(np.int16)  # NBR
    b4 = src.read(4, window=window).astype(np.int16)  # MNDWI
    b5 = src.read(5, window=window).astype(np.int16) if src.count >= 5 else None  # dNBR
    b6 = src.read(6, window=window).astype(np.int16) if src.count >= 6 else None  # RdNBR
    b7 = src.read(7, window=window).astype(np.int16) if src.count >= 7 else None  # OBS

    valid = (b1!=nd)&(b2!=nd)&(b3!=nd)&(b4!=nd)
    if b7 is not None:
        valid &= (b7!=nd)&(b7>=1)
    if not np.any(valid):
        return None

    ndvi  = (b1[valid].astype(np.float32))/10000.0
    evi   = (b2[valid].astype(np.float32))/10000.0
    nbr   = (b3[valid].astype(np.float32))/10000.0
    mndwi = (b4[valid].astype(np.float32))/10000.0

    # dNBR / RdNBR podem ter NoData próprios; filtra antes de escalar
    if b5 is not None:
        dmask = (b5[valid] != nd)
        dnbr  = (b5[valid][dmask].astype(np.float32))/10000.0
    else:
        dnbr = None
    if b6 is not None:
        rmask = (b6[valid] != nd)
        rdnbr = (b6[valid][rmask].astype(np.float32))/10000.0
    else:
        rdnbr = None

    obs = (b7[valid].astype(np.float32)) if b7 is not None else None
    return ndvi, evi, nbr, mndwi, dnbr, rdnbr, obs, valid.sum(), b1.size

def qc_one_month(yyyymm, target_samples=150_000, seed=123):
    fps = _list_month_files_merged_first(yyyymm)
    if not fps:
        return {"yyyymm": yyyymm, "n_files": 0, "note": "sem_arquivos"}

    rng = np.random.default_rng(seed)
    total_valid = 0
    total_px    = 0
    crs_set, res_set, nod_set, band_set = set(), set(), set(), set()

    for fp in fps:
        with rio.open(fp) as src:
            crs_set.add(str(src.crs))
            xres, yres = src.res
            res_set.add((float(xres), float(yres)))
            nod_set.add(src.nodata)
            band_set.add(src.count)
            for _, w in src.block_windows(1):
                blk = _iter_valid_block(src, w)
                if blk is None:
                    total_px += w.height * w.width
                    continue
                _, _, _, _, _, _, _, n_valid, n_tot = blk
                total_valid += n_valid
                total_px    += n_tot

    if total_valid == 0:
        return {"yyyymm": yyyymm, "n_files": len(fps), "note": "sem_validos",
                "crs": ";".join(sorted(crs_set)), "res": str(sorted(res_set)),
                "nodata": str(sorted(nod_set)), "bands": str(sorted(band_set))}

    p = min(1.0, target_samples / float(total_valid))

    all_ndvi, all_evi, all_nbr, all_mndwi = [], [], [], []
    all_dnbr, all_rdnbr, all_obs = [], [], []
    wmask_cnt = 0
    valid_cnt = 0
    nd_cnt    = 0

    for fp in fps:
        with rio.open(fp) as src:
            nd = src.nodata
            for _, w in src.block_windows(1):
                blk = _iter_valid_block(src, w)
                if blk is None:
                    nd_cnt += w.height * w.width
                    continue

                ndvi, evi, nbr, mndwi, dnbr, rdnbr, obs, n_valid, _ = blk
                valid_cnt += n_valid
                wmask_cnt += np.count_nonzero(mndwi > WATER_THR)

                # subamostragem
                n = ndvi.size
                k = int(np.round(p * n))
                if k > 0:
                    idx = rng.choice(n, size=min(k, n), replace=False)
                    all_ndvi.append(ndvi[idx])
                    all_evi.append(evi[idx])
                    all_nbr.append(nbr[idx])
                    all_mndwi.append(mndwi[idx])
                    if obs is not None and obs.size == n:
                        all_obs.append(obs[idx])

                if dnbr is not None and dnbr.size > 0:
                    k2 = int(np.round(p * dnbr.size))
                    if k2 > 0:
                        idx2 = rng.choice(dnbr.size, size=min(k2, dnbr.size), replace=False)
                        all_dnbr.append(dnbr[idx2])

                if rdnbr is not None and rdnbr.size > 0:
                    k3 = int(np.round(p * rdnbr.size))
                    if k3 > 0:
                        idx3 = rng.choice(rdnbr.size, size=min(k3, rdnbr.size), replace=False)
                        all_rdnbr.append(rdnbr[idx3])

    def _stats(arrs, name):
        if not arrs:
            return {f"{name}_p1": np.nan, f"{name}_p50": np.nan, f"{name}_p99": np.nan,
                    f"{name}_pct_zero": np.nan, f"{name}_n": 0}
        v = np.concatenate(arrs)
        if v.size == 0:
            return {f"{name}_p1": np.nan, f"{name}_p50": np.nan, f"{name}_p99": np.nan,
                    f"{name}_pct_zero": np.nan, f"{name}_n": 0}
        p1, p50, p99 = np.nanpercentile(v, [1, 50, 99])
        pct0 = 100 * np.mean(np.isclose(v, 0.0, atol=1e-6))
        return {f"{name}_p1": float(p1), f"{name}_p50": float(p50), f"{name}_p99": float(p99),
                f"{name}_pct_zero": float(pct0), f"{name}_n": int(v.size)}

    rec = {"yyyymm": yyyymm, "n_files": len(fps),
           "crs": ";".join(sorted(crs_set)),
           "res": str(sorted(res_set)),
           "nodata": str(sorted(nod_set)),
           "bands": str(sorted(band_set))}

    rec.update(_stats(all_ndvi, "NDVI"))
    rec.update(_stats(all_evi,  "EVI"))
    rec.update(_stats(all_nbr,  "NBR"))
    rec.update(_stats(all_mndwi,"MNDWI"))

    if all_dnbr:
        v = np.concatenate(all_dnbr)
        rec.update(_stats([v], "dNBR"))
        rec["dNBR_p95"] = float(np.nanpercentile(v, 95))
    else:
        rec.update({k: np.nan for k in ["dNBR_p1","dNBR_p50","dNBR_p99","dNBR_pct_zero","dNBR_n","dNBR_p95"]})

    if all_rdnbr:
        v = np.concatenate(all_rdnbr)
        rec.update(_stats([v], "RdNBR"))
    else:
        rec.update({k: np.nan for k in ["RdNBR_p1","RdNBR_p50","RdNBR_p99","RdNBR_pct_zero","RdNBR_n"]})

    if all_obs:
        o = np.concatenate(all_obs)
        rec.update({
            "OBS_min": float(np.nanmin(o)),
            "OBS_med": float(np.nanmedian(o)),
            "OBS_max": float(np.nanmax(o)),
            "OBS_n":   int(o.size)
        })
    else:
        rec.update({"OBS_min": np.nan, "OBS_med": np.nan, "OBS_max": np.nan, "OBS_n": 0})

    rec["pct_water_high_among_valid"] = 100.0 * (wmask_cnt / max(1, valid_cnt))
    rec["approx_pct_nodata_any"]     = 100.0 * (nd_cnt    / max(1, valid_cnt + nd_cnt))

    return rec

# ---------- descobrir automaticamente todos os YYYYMM disponíveis ----------
all_files = _glob_all("opt_*.tif")
all_ym = sorted({_extract_yyyymm(fp) for fp in all_files if _extract_yyyymm(fp)})
print(f"Meses detectados na pasta (serie completa): {len(all_ym)}")
print("Exemplos:", all_ym[:5], "…", all_ym[-5:])

# (Opcional) Filtrar por intervalo:
# all_ym = [ym for ym in all_ym if ym >= "198701" and ym <= "202112"]

# ---------- rodar QC ----------
rows=[]
for ym in tqdm(all_ym, desc="QC 300m — meses (arquivos mesclados)"):
    rows.append(qc_one_month(ym, target_samples=150_000, seed=123))

df = pd.DataFrame(rows).sort_values("yyyymm")
display_cols = [
    "yyyymm","n_files","approx_pct_nodata_any","pct_water_high_among_valid",
    "NDVI_p1","NDVI_p50","NDVI_p99",
    "EVI_p1","EVI_p50","EVI_p99",
    "NBR_p1","NBR_p50","NBR_p99",
    "MNDWI_p1","MNDWI_p50","MNDWI_p99",
    "dNBR_p50","dNBR_p95","RdNBR_p50",
    "OBS_min","OBS_med","OBS_max"
]
print("— Resumo (amostra) —")
print(df[display_cols].head(10).to_string(index=False))

qc_csv = OUT_DIR / "qc_optical_300m_summary.csv"
df.to_csv(qc_csv, index=False)
print("✔ QC salvo em:", qc_csv)



Meses detectados na pasta (serie completa): 197
Exemplos: ['198509', '198603', '198607', '198609', '198611'] … ['202103', '202105', '202107', '202109', '202111']


QC 300m — meses (arquivos mesclados):   0%|          | 0/197 [00:00<?, ?it/s]

— Resumo (amostra) —
yyyymm  n_files  approx_pct_nodata_any  pct_water_high_among_valid  NDVI_p1  NDVI_p50  NDVI_p99   EVI_p1  EVI_p50  EVI_p99    NBR_p1  NBR_p50  NBR_p99  MNDWI_p1  MNDWI_p50  MNDWI_p99  dNBR_p50  dNBR_p95  RdNBR_p50  OBS_min  OBS_med  OBS_max
198509        1              41.957689                         0.0   0.2318    0.4732  0.790500 0.141500   0.2922 0.556601 -0.071700   0.3194 0.718200   -0.5773    -0.4498  -0.021099    0.0735    0.2750     0.1221      1.0      2.0      8.0
198603        1              41.091803                         0.0   0.4492    0.6336  0.811500 0.153000   0.3415 0.531200  0.216400   0.5522 0.765000   -0.5941    -0.4825  -0.034700    0.0167    0.1001     0.0216      1.0      5.0     19.0
198607        1              41.188805                         0.0   0.2819    0.5106  0.741900 0.125100   0.2799 0.489200 -0.080300   0.3426 0.729000   -0.5949    -0.5014  -0.061200    0.1480    0.3280     0.2145      1.0      2.0     10.0
198609        1